<a href="https://colab.research.google.com/github/hangatzu2017/RAG/blob/main/RAG_Chromadb_mistral7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG con Minstral 7B

In questo notebook esploreremo l'utilizzo del modello open source **Mistral-7B** facendo uso dei transformers di Hugging Face e del framework Langchain per la realizzazione di un sistema RAG basico.
Per la gestione dei vettori invece ci affideremo al Database vettoriale opensource ChromaDB.

---

🚨 _Nota Bene: che l'esecuzione su CPU è molto lenta. Se si esegue su Google Colab, è possibile evitare questo problema andando su **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**._

---

Iniziamo con il verificare che il runtime sia impostato correttamente con:

In [1]:
!nvidia-smi

Fri Dec 22 15:04:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Proseguiamo con l'installazione di tutte le librerie necessarie tramite pip con `pip install`.

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install -q -U chromadb

Importiamo le necessarie librerie

In [3]:
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

Scarichiamo un file (PDF nel nostro caso) dal quale il nostro LLM acquisirà la "conoscenza" (knowledge).
Personalmente ho usato questo PDF: https://www.maristi.it/ciao/wp-content/uploads/2020/02/scuolaguida-manuale-teoria-A1-A-B.pdf poichè sufficientemente ampio per testare con un bel po' di domande. Siete comunque liberi di caricare qualsiasi file desideriate.

In [4]:
loader_doc = PyPDFLoader("https://www.maristi.it/ciao/wp-content/uploads/2020/02/scuolaguida-manuale-teoria-A1-A-B.pdf")
documents_second = loader_doc.load()
text_splitter_doc = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts_doc = text_splitter_doc.split_documents(documents_second)

print(texts_doc[0].page_content)

Valerio Platia 
Roberto Mastri 
 
 
 
Manuale di  teoria 
per le patenti A1, A e  B
Edizione 2008


Procediamo con la creazione degli embeddings (vettori n dimensionali) che rappresentano il nostro testo diviso in parti (chunks). I modelli di embeddings sono decisamente molti, per l'italiano però (come al solito) non vi è molto materiale sopratutto open source, pertanto consiglio di deviare verso modelli multi language che tutto sommato non performano malaccio.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

model_name = "intfloat/multilingual-e5-base"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Vettorializzazione dei chunk calcolati sopra e salvataggio (per la persistenza) nella cartella "chroma_db"
vectordb = Chroma.from_documents(documents=texts_doc, embedding=embeddings, persist_directory="chroma_db")

# Impostazione del retriever
retriever = vectordb.as_retriever()


Definiamo ora il prompt di Sistema, ossia diamo un'identità al nostro modello LLM ed impostiamo alcuni parametri di base per lo stesso. Uno dei parametri a cui fare attenzione è la temperatura, per i sistemi RAG infatti è bene tenerla più bassa possibile (vicino allo zero) cosicchè le risposte siano attinenti ai fatti di contesto evitando le famose "allucinazioni" tipiche dei modelli di grandi dimensioni.

----

_Nota Bene: Personalmente preferisco che il prompt di sistema resti in inglese, sembra che in questo modo venga compreso meglio dal modello, ma predentelo con le pinze, specificherò comunque che la sua risposta dovrà sempre essere data in lingua italiana_

----



In [6]:
from langchain.llms import CTransformers

# Configurazione parametri base LLM
config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 2048, 'temperature':0, 'gpu_layers':50}

# Utilizziamo il modello quantizzato a 4bit (formato GGUF)
llm = CTransformers(model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",gpu_layers=50, config=config)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Definiamo il template del prompt di sistema (in inglese specificando che desideriamo risultati in italiano)
template = """<s>[INST] You are a helpful, respectful and honest assistant. Your answers should always be in Italian. Answer exactly in few words from the context.
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])

# Impostazione della catena di Question and Answer (approfondisci qui : https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html#)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={"prompt": prompt_template},
)


In [11]:
def run_my_rag(qa, query):
    print(f"Domanda: {query}\n")
    result = qa.run(query)
    print("\nRisposta: ", result)


In [12]:
### Facciamo la nostra domanda
query =""" Quando è vietato il sorpasso? """
run_my_rag(qa, query)

Domanda:  Quando è vietato il sorpasso? 



> Entering new RetrievalQA chain...

> Finished chain.

Risposta:  
Il sorpasso è consentito solo se il veicolo che si sorpassa è un motociclo o una bicicletta, oppure se l'incrocio è regolato da segnalazione semaforica o da agenti preposti al traffico e non invade l'opposto senso di marcia.
